In [2]:
import pandas as pd

In [3]:

df = pd.read_hdf("./retention_features.h5")
print(df.columns)

Index(['user_id', 'card_id', 'card_text', 'is_new_fact',
       'user_n_study_positive', 'user_n_study_negative', 'user_n_study_total',
       'card_n_study_positive', 'card_n_study_negative', 'card_n_study_total',
       'usercard_n_study_positive', 'usercard_n_study_negative',
       'usercard_n_study_total', 'acc_user', 'acc_card', 'acc_usercard',
       'usercard_delta', 'usercard_delta_previous',
       'usercard_previous_study_response', 'leitner_box', 'sm2_efactor',
       'sm2_interval', 'sm2_repetition', 'delta_to_leitner_scheduled_date',
       'delta_to_sm2_scheduled_date', 'repetition_model',
       'elapsed_milliseconds', 'correct_on_first_try', 'utc_datetime',
       'utc_date', 'label', 'debug_id', 'n_minutes_spent'],
      dtype='object')


In [4]:

df.rename(columns = {'label':'response'}, inplace = True)

In [ ]:
import uuid
# Adapted from 
# https://stackoverflow.com/questions/66366449/how-to-generate-8-digit-unique-identifier-to-replace-the-existing-one-in-python
original_ids = df['user_id'].unique()
new_ids = {cid: uuid.uuid4().int for cid in original_ids}
df['Anon Student Id'] = df['user_id'].map(new_ids)

In [ ]:
df.drop(['C'], inplace=True, axis=1)